## Indexing of Steinmetz data

In [1]:
#@title Data retrieval
import os, requests
from pathlib import Path
DATA_DIR = Path.cwd().parent / 'data'

fname = []
for j in range(3):
  fname.append(DATA_DIR / f"steinmetz_part{j}.npz")
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not fname[j].exists():
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

In [2]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load(DATA_DIR / f"steinmetz_part{j}.npz", allow_pickle=True)['dat']))

In [3]:
from brain_areas import *

# Regions in Figure 3 with >= ~10% of visually-selective neurons
VISUAL_AREAS = [
    AREA_VISp,
    AREA_VISl,
    AREA_VISpm,
    AREA_VISam,
    AREA_CP,
    AREA_LD,
    AREA_SCs,
]


selectors = []
for dat in alldat:
    sel = {
        "NEURON_VISUAL": np.isin(dat['brain_area'], VISUAL_AREAS),
        "CHOICE_RIGHT": dat['response'] == -1,
        "CHOICE_LEFT": dat['response'] == 1,
        "CHOICE_NONE": dat['response'] == 0,
        "STIM_RIGHT": dat['contrast_right'] > dat['contrast_left'],
        "STIM_LEFT": dat['contrast_right'] < dat['contrast_left'],
        "STIM_EQUAL": (dat['contrast_right'] == dat['contrast_left']) & (dat['contrast_right'] > 0),
        "STIM_NONE": (dat['contrast_right'] == dat['contrast_left']) & (dat['contrast_right'] == 0),
        "STIM_RIGHT_HIGH": dat['contrast_right'] == 1,
        "STIM_RIGHT_MEDIUM": dat['contrast_right'] == 0.5,
        "STIM_RIGHT_LOW": dat['contrast_right'] == 0.25,
        "STIM_RIGHT_NONE": dat['contrast_right'] == 0,
    }
    sel.update({
        "CHOICE_CORRECT": (
            (sel["STIM_RIGHT"] & sel["CHOICE_RIGHT"])
            | (sel["STIM_LEFT"] & sel["CHOICE_LEFT"])
            | (sel["STIM_NONE"] & sel["CHOICE_NONE"])
        ),
        "CHOICE_MISS": ~sel["STIM_NONE"] & sel["CHOICE_NONE"]
    })
    selectors.append(sel)

In [4]:
# for i in range(len(alldat)):
#     sel = selectors[i]
#     print(alldat[i]['spks'][sel["NEURON_VISUAL"]][:, sel["STIM_RIGHT_HIGH"] & sel["CHOICE_LEFT"]].shape)

(178, 11, 250)
(533, 8, 250)
(228, 7, 250)
(39, 3, 250)
(0, 4, 250)
(0, 6, 250)
(101, 3, 250)
(89, 8, 250)
(221, 5, 250)
(204, 17, 250)
(275, 10, 250)
(145, 11, 250)
(66, 6, 250)
(79, 9, 250)
(0, 15, 250)
(0, 14, 250)
(12, 2, 250)
(158, 3, 250)
(179, 8, 250)
(122, 5, 250)
(100, 1, 250)
(133, 18, 250)
(190, 3, 250)
(146, 7, 250)
(94, 8, 250)
(162, 4, 250)
(127, 6, 250)
(71, 1, 250)
(340, 1, 250)
(0, 3, 250)
(0, 4, 250)
(73, 5, 250)
(383, 4, 250)
(0, 7, 250)
(75, 5, 250)
(53, 11, 250)
(92, 9, 250)
(0, 10, 250)
(141, 1, 250)


In [6]:
with open(DATA_DIR / "selectors.npy", "wb") as f:
    np.save(f, selectors)